In [ ]:
# Install and Import all required libraries

%pip install openai pandas cassandra-driver
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
from getpass import getpass
import openai
import numpy
import pandas as pd
import os

In [ ]:
# Collect all information required to connect to your Astra Database and OpenAI API. Also specify the model_id for generating the embeddings.

ASTRA_DB_SECURE_BUNDLE_PATH = input("Please provide the full path to your Secure Connect Bundle zipfile: ")
ASTRA_DB_APPLICATION_TOKEN = getpass("Please provide your Database Token ('AstraCS:...' string): ")
ASTRA_DB_KEYSPACE = input("Please provide the Keyspace name for your Database: ")
OPENAI_API_KEY = getpass("Please enter your OpenAI API Key: ")
model_id = "text-embedding-ada-002"

In [ ]:
# Connect to your Astra Database

cluster = Cluster(
    cloud={
        "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH
    },
    auth_provider=PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN),
)

session = cluster.connect()
session.set_keyspace(ASTRA_DB_KEYSPACE)


In [ ]:
# Create the table to store recipes
keyspace = ASTRA_DB_KEYSPACE
analyzer = {"index_analyzer" : "standard"}

session.execute(f"""CREATE TABLE IF NOT EXISTS {keyspace}.recipes_table (
    recipe_id int,
    recipe_type text,
    recipe_description text,
    ingredients text,
    instructions text,
    recipe_vector vector<float, 1536>,
    PRIMARY KEY (recipe_id))"""
)

# Create the indexes for the table, including the index for the recipe_vector column and the ingredients column.
# The ingredients column will use the standard analyzer, which will tokenize the ingredients text.
session.execute(f"""CREATE CUSTOM INDEX IF NOT EXISTS recipe_vec_idx ON {keyspace}.recipes_table (recipe_vector) USING 'StorageAttachedIndex'""")
session.execute(f"""CREATE CUSTOM INDEX IF NOT EXISTS ingredients_idx ON {keyspace}.recipes_table (ingredients) USING 'StorageAttachedIndex' WITH OPTIONS = {analyzer}""")

In [ ]:
# Read in the CSV file containing the recipes.

recipes = pd.read_csv('CookingSuggestions/recipe_data/recipie_en.csv')
recipes

In [ ]:
# Insert the recipes into the table, and generate the recipe_vector for each recipe using the OpenAI API.

openai.api_key = OPENAI_API_KEY
for id, row in recipes.iterrows():
    recipe_id = row['RecipeID']
    recipe_type = row['Type']
    recipe_description = row['Description']
    ingredients = row['Ingredients']
    instructions = row['Instructions']
    recipe_vsource = f'{recipe_description}, which is a type of {recipe_type}, is made of {ingredients} by following the these cooking instructions: {instructions}'
    recipe_vector = openai.Embedding.create(input=recipe_vsource, model=model_id)['data'][0]['embedding']
    session.execute(f"""INSERT INTO {keyspace}.recipes_table (recipe_id, recipe_type, recipe_description, ingredients, instructions, recipe_vector)
                        VALUES ({recipe_id}, '{recipe_type}', '{recipe_description}', '{ingredients}', '{instructions}', {recipe_vector})""")

In [ ]:
# Ask some questions to the user to get the meal preference and ingredient for the next meal to be cooked.

meal_preference = input("What type of meal would you like to cook? ")
ingredient_input = input("Please specify an ingredient that you want to use for cooking your next meal: ")

embedding = openai.Embedding.create(input=meal_preference, model=model_id)['data'][0]['embedding']

In [ ]:
# The query below will return the top 5 recipes that are most similar to the meal preference and ingredient specified by the user.
# The query uses the ANN index on the recipe_vector column to find the most similar recipes, and the ingredients column to filter the results.
# The query also uses the ORDER BY clause to sort the results by the similarity score, and the LIMIT clause to return only the top 5 results.

query = SimpleStatement(
    f"""
    SELECT *
    FROM {keyspace}.recipes_table
    WHERE ingredients : '{ingredient_input}'
    ORDER BY recipe_vector ANN OF {embedding} LIMIT 5;
    """
    )
display(query)

In [ ]:
# Execute the query and display the results.
# If no results are returned, then display a message to the user to try again.

recipes_to_cook = session.execute(query)
if not the_recipes:
    print("There were no recipes found for your search criteria. Please try again.")
else:
    the_recipes = recipes_to_cook._current_rows
    print(f"""You can cook the following recipes with {ingredient_input}: \n""")
    for row in the_recipes:
        print(f"""{row.recipe_description} ({row.recipe_type}) using {row.ingredients}, with the following cooking instructions: {row.instructions} \n""")